In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_val_predict, RandomizedSearchCV
from sklearn.ensemble import HistGradientBoostingClassifier, StackingClassifier
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector

from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

# Disable warnings
import warnings
warnings.filterwarnings("ignore")

# Palette
sns.set_palette("icefire")

# Load and Overview


In [ ]:
train = pd.read_csv("kaggle/input/playground-series-s5e11/train.csv", index_col=0)
test = pd.read_csv("kaggle/input/playground-series-s5e11/test.csv", index_col=0)
# orig = pd.read_csv("/kaggle/input/loan-prediction-dataset-2025/loan_dataset_20000.csv")

for col in train.select_dtypes(include="object").columns:
            train[col] = train[col].astype("category")
for col in test.select_dtypes(include="object").columns:
            test[col] = test[col].astype("category")
    
print(
    "Missing Train: ",
    train.isnull().sum().sum(),
    " - Missing Test: ",
    test.isnull().sum().sum(),
    end="\n\n",
)
train.head(3)

In [ ]:
train.describe(include="all")

# Distribution

In [ ]:
train.info()

In [ ]:
for col in train.select_dtypes(include="category").columns.to_list() + ["loan_paid_back"]:
    sns.countplot(data=train, x=train[col], palette="icefire")
    plt.xticks(rotation=45, ha="right")
    plt.xlabel(col)
    plt.ylabel("Count")
    plt.show()

In [ ]:
class Features(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.risk = None

    def fit(self, df, y=None):
        df = df.copy()
        if isinstance(y, pd.Series):
            y = y.copy()
        else:
            y = pd.Series(data=y, index=df.index, name="y")

        if y is not None:
            df["loan_paid_back"] = y.values
            self.risk = df.groupby("employment_status")["loan_paid_back"].mean()

        self.debt_to_income_ratio_mean = df["debt_to_income_ratio"].mean()

        return self

    def transform(self, df):
        df = df.copy()

        df["annual_incomeXcredit_score"] = df["annual_income"] * df["credit_score"]
        df["loan_to_income"] = df["loan_amount"] / (df["annual_income"] + 1)
        df["interest_burden"] = (df["loan_amount"] * df["interest_rate"]) / (
            df["annual_income"] + 1
        )
        df["log_income"] = np.log1p(df["annual_income"])
        df["log_loan_amount"] = np.log1p(df["loan_amount"])

        df["debt_to_income_ratio_diff"] = (
            df["debt_to_income_ratio"] - self.debt_to_income_ratio_mean
        )
        df["debt_to_income_ratio_norm"] = (
            df["debt_to_income_ratio"] / self.debt_to_income_ratio_mean
        )

        if self.risk is not None:
            df["risk_map"] = df["employment_status"].map(self.risk).astype(float)

        df["credit_dti_interaction"] = df["credit_score"] / (
            df["debt_to_income_ratio"] + 1
        )
        df["income_dti_interaction"] = df["annual_income"] / (
            df["debt_to_income_ratio"] + 1
        )

        df = df.drop(
            [
                "loan_purpose",
                "gender",
                "education_level",
                "marital_status",
            ],
            axis=1,
        )

        return df

In [ ]:
columnTransformer = ColumnTransformer(
    [
        (
            "int",
            Pipeline([("scaler", MinMaxScaler())]),
            make_column_selector(dtype_exclude="category"),
        ),
        (
            "cat",
            Pipeline(
                [
                    (
                        "ordinal",
                        OrdinalEncoder(
                            handle_unknown="use_encoded_value", unknown_value=-1
                        ),
                    )
                ]
            ),
            make_column_selector(dtype_include="category"),
        ),
    ]
).set_output(transform="pandas")

In [ ]:
X = train.iloc[:, :-1]
y = train.iloc[:, -1]
X_pred = test

FeatureFrame = Pipeline(
    [
        ("feature", Features()),
        ("transformer", columnTransformer),
    ]
).fit_transform(X, y)
catFeatures = FeatureFrame.select_dtypes(include="category").columns.to_list()
numFeatures = FeatureFrame.select_dtypes(exclude="category").columns.to_list()
pd.concat([FeatureFrame, y],axis=1).to_csv("kaggle/working/" + "FeatureFrame.csv")
FeatureFrame.info()

In [ ]:
xgb_params_search = {
    "xgb__booster": ["gbtree", "dart"],
    "xgb__n_estimators": range(500, 5001, 500),
    "xgb__learning_rate": [0.1, 0.01, 0.001, 0.3, 0.03, 0.003],
    "xgb__max_depth": range(3, 12, 2),
    "xgb__min_child_weight": range(1, 10, 2),
    "xgb__gamma": range(0, 10, 1),
    "xgb__subsample": [0.5, 0.7, 0.9],
    "xgb__colsample_bytree": [0.5, 0.7, 0.9],
}
cat_params_search = {
    "cat__n_estimators": range(500, 5001, 500),
    "cat__learning_rate": [0.1, 0.01, 0.001, 0.3, 0.03, 0.003],
    "cat__depth": range(4, 11, 2),
    "cat__l2_leaf_reg": range(1, 10, 2),
    "cat__bagging_temperature": [0, 1],
    "cat__rsm": [0.5, 0.7, 0.9],
    "cat__bootstrap_type": ["Bayesian", "Bernoulli", "MVS"],
}
lgb_params_search = {
    "lgb__n_estimators": range(500, 5001, 500),
    "lgb__learning_rate": [0.1, 0.01, 0.001, 0.3, 0.03, 0.003],
    "lgb__num_leaves": range(16, 257, 16),
    "lgb__max_depth": range(3, 15, 2),
    "lgb__min_child_samples": range(5, 26, 10),
    "lgb__feature_fraction": [0.6, 0.8, 1.0],
    "lgb__bagging_fraction": [0.6, 0.8, 1.0],
    "lgb__bagging_freq": range(0, 10, 3),
    "lgb__min_split_gain": range(0, 10, 3),
    "lgb__lambda_l1": [0, 0.1, 0.5, 1, 5],
    "lgb__lambda_l2": [0, 0.1, 0.5, 1, 5],
    "lgb__boosting_type": ["gbdt", "goss", "dart"],
}
hgb_params_search = {
    "hgb__max_iter": range(500, 5001, 500),
    "hgb__learning_rate": [0.1, 0.01, 0.001, 0.3, 0.03, 0.003],
    "hgb__max_depth": range(3, 15, 2),
    "hgb__min_samples_leaf": range(5, 100, 30),
    "hgb__l2_regularization": range(1, 10, 2),
}

In [ ]:
cv = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

xgb = Pipeline([
    ("feature", Features()),
    ("xgb",XGBClassifier(
        random_state=42,
        n_estimators=1500,
        learning_rate=0.03,
        enable_categorical=True,
        verbose=False,
        device="cuda",
        n_job=-1,
    )),
])

cat = Pipeline([
    ("feature", Features()),
    ("cat", CatBoostClassifier(
        random_state=42,
        n_estimators=1500,
        learning_rate=0.03,
        cat_features=catFeatures,
        verbose=False,
        task_type="GPU",
    )),
])

lgb = Pipeline([
    ("feature", Features()),
    ("lgb", LGBMClassifier(
        random_state=42, 
        n_estimators=1500, 
        learning_rate=0.03, 
        verbose=-1,
        device="gpu",
        n_job=-1,
    )),
])

hgb = Pipeline([
    ("feature", Features()),
    ("transformer", columnTransformer),
    ("hgb", HistGradientBoostingClassifier(
        random_state=42, 
        max_iter=500, 
        learning_rate=0.03, 
        verbose=0
    )),
])

In [ ]:
xgb_search = RandomizedSearchCV(
    xgb,
    param_distributions=xgb_params_search,
    n_iter=50,
    scoring="roc_auc",
    cv=cv,
    verbose=1,
    n_jobs=-1,
).fit(X, y)

cat_search = RandomizedSearchCV(
    cat,
    param_distributions=cat_params_search,
    n_iter=50,
    scoring="roc_auc",
    cv=cv,
    verbose=1,
    n_jobs=-1,
).fit(X, y)

lgb_search = RandomizedSearchCV(
    lgb,
    param_distributions=lgb_params_search,
    n_iter=50,
    scoring="roc_auc",
    cv=cv,
    verbose=1,
    n_jobs=-1,
).fit(X, y)

hgb_search = RandomizedSearchCV(
    hgb,
    param_distributions=hgb_params_search,
    n_iter=50,
    scoring="roc_auc",
    cv=cv,
    verbose=1,
    n_jobs=-1,
).fit(X, y)

In [ ]:
pd.Series(xgb_search.best_params_).to_csv("kaggle/working/best/" + "xgb_params.csv")
pd.Series(cat_search.best_params_).to_csv("kaggle/working/best/" + "cat_params.csv")
pd.Series(lgb_search.best_params_).to_csv("kaggle/working/best/" + "lgb_params.csv")
pd.Series(hgb_search.best_params_).to_csv("kaggle/working/best/" + "hgb_params.csv")

In [ ]:
xgb.set_params(
    **pd.read_csv("kaggle/working/best/" + "xgb_params.csv", index_col=0)
    .squeeze()
    .to_dict()
)
cat.set_params(
    **pd.read_csv("kaggle/working/best/" + "cat_params.csv", index_col=0)
    .squeeze()
    .to_dict()
)
lgb.set_params(
    **pd.read_csv("kaggle/working/best/" + "lgb_params.csv", index_col=0)
    .squeeze()
    .to_dict()
)
hgb.set_params(
    **pd.read_csv("kaggle/working/best/" + "hgb_params.csv", index_col=0)
    .squeeze()
    .to_dict()
)

In [ ]:
score_xgb = cross_val_score(xgb, X, y, cv=cv, scoring="roc_auc")
print(
    f"xgb scores across folds: {score_xgb} - Mean Score: {score_xgb.mean():0.4f}"
)

score_cat = cross_val_score(cat, X, y, cv=cv, scoring="roc_auc")
print(
    f"cat scores across folds: {score_cat} - Mean Score: {score_cat.mean():0.4f}"
)

score_lgb = cross_val_score(lgb, X, y, cv=cv, scoring="roc_auc")
print(
    f"lgb scores across folds: {score_lgb} - Mean Score: {score_lgb.mean():0.4f}"
)
score_hgb = cross_val_score(hgb, X, y, cv=cv, scoring="roc_auc")
print(
    f"hgb scores across folds: {score_hgb} - Mean Score: {score_hgb.mean():0.4f}"
)


print(f"Overall Performance: {np.mean([score_xgb, score_cat, score_lgb, score_hgb], axis=0)}")

In [ ]:
total = score_xgb.mean() + score_cat.mean() + score_lgb.mean() + score_hgb.mean()
xgb_weight = score_xgb.mean() / total
cat_weight = score_cat.mean() / total
lgb_weight = score_lgb.mean() / total
hgb_weight = score_hgb.mean() / total
print(xgb_weight, cat_weight, lgb_weight, hgb_weight)

In [ ]:
xgb.fit(X, y)
cat.fit(X, y)
lgb.fit(X, y)
hgb.fit(X, y)


In [ ]:
importantFeatures = pd.DataFrame(
    data={
        "xgb": xgb.named_steps["xgb"].feature_importances_ / xgb.named_steps["xgb"].feature_importances_.max(),
        "cat": cat.named_steps["cat"].feature_importances_ / cat.named_steps["cat"].feature_importances_.max(),
        "lgb": lgb.named_steps["lgb"].feature_importances_ / lgb.named_steps["lgb"].feature_importances_.max(),
    },
    index = FeatureFrame.columns
)
importantFeatures["Overall"] = importantFeatures.mean(axis=1)
importantFeatures = (importantFeatures.sort_values("Overall", ascending=False) * 100).round(2)
importantFeatures

In [ ]:
xgb_pred = cross_val_predict(xgb, X, y, cv=cv, method="predict_proba")

In [ ]:
treePredictions = pd.DataFrame(
    data={
        "xgbPrediction": cross_val_predict(xgb, X, y, cv=cv, method="predict_proba")[:,1],
        "catPrediction": cross_val_predict(cat, X, y, cv=cv, method="predict_proba")[:,1],
        "lgbPrediction": cross_val_predict(lgb, X, y, cv=cv, method="predict_proba")[:,1],
        "hgbPrediction": cross_val_predict(hgb, X, y, cv=cv, method="predict_proba")[:,1],
    },
    index=FeatureFrame.index,
)

treePredictions["WeightedPrediction"] = (
    treePredictions["xgbPrediction"] * xgb_weight
    + treePredictions["catPrediction"] * cat_weight
    + treePredictions["lgbPrediction"] * lgb_weight
    + treePredictions["hgbPrediction"] * hgb_weight
)
treePredictions.head(10)

In [ ]:
ensemblePredictions = pd.DataFrame(
    data={
        "xgbPrediction": xgb.predict_proba(X_pred)[:,1],
        "catPrediction": cat.predict_proba(X_pred)[:,1],
        "lgbPrediction": lgb.predict_proba(X_pred)[:,1],
    },
    index=X_pred.index,
)
ensemblePredictions["Loan_paid_back"] = (ensemblePredictions["xgbPrediction"] * xgb_weight + ensemblePredictions["catPrediction"] * cat_weight + ensemblePredictions["lgbPrediction"] * lgb_weight).round(0)

In [ ]:
ensemblePredictions["Loan_paid_back"].to_csv("kaggle/working" + "submission.csv")

In [ ]:
treePredictions.round(0).sum()

In [ ]:
treePredictions.corr()